# Imports and settings

In [235]:
#from elasticsearch_dsl import Search, Q, Range, connections
#from elasticsearch.client import Elasticsearch
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
from openai import OpenAI
import numpy as np

In [236]:
DATA_FOLDER = "../data/events/browse_ai/" 

In [237]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)
pd.set_option('display.max_colwidth', 300)

In [238]:
df = pd.read_pickle(f"{DATA_FOLDER}df_browse_ai before description 2024-12-19_18-27-21.pkl")

In [ ]:
len(df)

In [ ]:
df.replace("nan", np.nan, inplace=True)

In [241]:
extracted_columns = [col for col in df.columns if col.startswith("extr")]
required_cols = ["Event_URL", "isOnMultipleDays"]
required_cols.extend(extracted_columns)
required_cols_except_start_date = required_cols.copy()
required_cols_except_start_date.remove("extrStartDate")

In [242]:
df['count_notna'] = df[required_cols].notna().sum(axis=1)

In [243]:
df = df.loc[df.groupby('Event_URL')['count_notna'].idxmax()]

In [ ]:
len(df)

### manual data replacements

In [ ]:
flt = df["Event_description"].str.contains("Caspar David Friedrich. Wo alles begann", na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp

df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrStreetHouseNumber"]="Tzschirnerpl. 2"
df.loc[id, "extrMultipleDaysDetails"] = """Freitag-Montag, Mittwoch 10-18 Uhr Donnerstag 10-21 Uhr Dienstag geschlossen"""



In [ ]:
flt = df["Event_description"].str.contains("Napoleon mit seiner Grande Armée marschierte am 24.", na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-27"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Militärhistorisches Museum der Bundeswehr"
df.loc[id, "extrStreetHouseNumber"]="Olbrichtpl. 2"
df.loc[id, "extrLocationDetails"]="Olbrichtpl. 2, 01099 Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Montag 10 - 21 Uhr Dienstag, Donnerstag, Freitag, Samstag, Sonntag 10-18 Uhr Mittwoch geschlossen"""


In [ ]:
flt = df["Event_description"].str.contains("Wer entscheidet eigentlich, wer dazugehört", na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Robotron-Kantine in Dresden"
df.loc[id, "extrStreetHouseNumber"]="Zinzendorfstraße 5"
df.loc[id, "extrLocationDetails"]="Zinzendorfstraße 5, 01069 Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Mi – Fr 16 – 19 Uhr Sa & So und Feiertag 12 -18 Uhr"""


In [ ]:
flt = df["Event_description"].str.contains("Eine Ausstellung zeitgenössischer Kunst zu den Auswirkungen rassistischer", na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Robotron-Kantine in Dresden"
df.loc[id, "extrStreetHouseNumber"]="Zinzendorfstraße 5"
df.loc[id, "extrLocationDetails"]="Zinzendorfstraße 5, 01069 Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Mi – Fr 16 – 19 Uhr Sa & So und Feiertag 12 -18 Uhr"""


In [ ]:
flt = df["Event_description"].str.contains('Unter dem Motto "Über die Natur" findet zum vierten Mal ', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Schloss Pillnitz"
df.loc[id, "extrLocationDetails"]="August-Böckstiegel-Straße 2, 01326 Dresden"
df.loc[id, "extrStreetHouseNumber"]="August-Böckstiegel-Straße 2"
df.loc[id, "extrMultipleDaysDetails"] = """täglich 10-17 Uhr, Montag geschlossen"""


In [ ]:
flt = df["Event_description"].str.contains('Mikroschnitzereien auf Streichhölzern oder in Walnussschalen, Modelle und Puppenspielzeug', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-20"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="Köpckestraße 1, 01097 Dresden"
df.loc[id, "extrStreetHouseNumber"]="Köpckestraße 1"
df.loc[id, "extrMultipleDaysDetails"] = """täglich 10-18 Uhr, Montag geschlossen"""


In [ ]:
flt = df["Event_description"].str.contains('Dem langjährigen Professor für Monumentalmalerei widmet sich die neue Sonderausstellung', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Schloss Pillnitz"
df.loc[id, "extrLocationDetails"]="August-Böckstiegel-Straße 2, 01326 Dresden"
df.loc[id, "extrStreetHouseNumber"]="August-Böckstiegel-Straße 2"
df.loc[id, "extrMultipleDaysDetails"] = """Dienstag bis Sonntag 10 bis 17 Uhr"""


In [ ]:
flt = df["Event_description"].str.contains('Tonband- und Kassettengeräte kam man', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-19"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]=" Städtische Galerie Dresden"
df.loc[id, "extrMultipleDaysDetails"] = """Eröffnung: DO 19.09. 19.00 Uhr DI bis SO 10.00 bis 18.00 Uhr FR 10.00 bis 19.00 Uhr"""


In [ ]:
flt = df["Event_description"].str.contains('100 Jahre KUNSTAUSSTELLUNG KÜHL in Dresden.', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]=" Kunstausstellung Kühl Dresden"
df.loc[id, "extrStreetHouseNumber"]="Nordstraße 5"
df.loc[id, "extrMultipleDaysDetails"] = """Mi bis Fr 11 –19 + Sa 11 –16 Uhr"""


In [ ]:
flt = df["Event_description"].str.contains('Auf die Elbe, fertig, los!', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Blaues Wunder"
df.loc[id, "extrStartTime"] = "11"
df.loc[id, "extrEndTime"] = "15"

In [ ]:
flt = df["Event_description"].str.contains('Das Highlight für Kinder ist die Parkeisenbahn.', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-26"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Parkeisenbahn Dresden"
df.loc[id, "extrLocationDetails"]="01219 Dresden-Altstadt"
df.loc[id, "extrMultipleDaysDetails"] = """Mittwoch bis Freitag: 13:00 bis 18:00 Uhr Samstag, Sonntag, Feiertag: 10:30 bis 18:00 Uhr"""

In [ ]:
flt = df["Event_description"].str.contains('Süße, halb tote Katze verschwunden! Ihre Spur', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Technische Sammlungen Dresden"
df.loc[id, "extrLocationDetails"]="01277 Dresden Junghannsstraße 1-3"
df.loc[id, "extrStreetHouseNumber"]="Junghannsstraße 1-3"
df.loc[id, "extrMultipleDaysDetails"] = """DI bis FR 9.00 bis 17.00 Uhr SA/SO 10.00 bis 18.00 Uhr"""

In [ ]:
flt = df["Event_description"].str.contains('prägenden Periode in der Geschichte des DHMD und stellt sie in den Kontext der wohl einschneidendsten Umbrüche der', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationName"]="Hygienemuseum"
df.loc[id, "extrLocationDetails"]="01069 Dresden Lingnerplatz 1"
df.loc[id, "extrStreetHouseNumber"]="Lingnerplatz 1"
df.loc[id, "extrMultipleDaysDetails"] = """Dienstag bis Sonntag, Feiertage: 10 bis 18 Uhr"""

In [ ]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Kinderbiennale-Planet-Utopia-Dresden-114311.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="01097 Dresden Palaisplatz 11"
df.loc[id, "extrStreetHouseNumber"]="Palaisplatz 11"
df.loc[id, "extrMultipleDaysDetails"] = "Diensatg bis Sonntag 10 bis 18 Uhr"

In [ ]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Wiederentdeckt-Dresdner-Malerinnen-Der-Romantik-Dresden-116639.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="01097 Dresden Hauptstr. 13"
df.loc[id, "extrMultipleDaysDetails"] = "MI bis FR 10.00 bis 17.00 Uhr SA/SO 12.00 bis 17.00 Uhr"

In [ ]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Von-Wanderlust-Und-Reisefrust', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-27"
df.loc[id, "extrCity"]="Dresden"
df.loc[id, "extrLocationDetails"]="01099 Dresden Nordstraße 28"
df.loc[id, "extrMultipleDaysDetails"] = "MI bis SO 12.00 bis 17.00 Uhr"

In [ ]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Ausstellung-Home-Sweet-Home-Danny-Hermann-Radebeul-117962.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-07"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrMultipleDaysDetails"] = "Samstag & Sonntag 14-18 Uhr"

In [ ]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Sonderausstellung-Andreas-Braeunsdorf-Malerei-Dresden-116638.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-07"
df.loc[id, "extrEndDate"] = "2024-09-29"

In [ ]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Udo-Zimmermann-Ein-Moderner-Romantiker-Dresden-114359.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-09-29"
df.loc[id, "extrMultipleDaysDetails"] = "MI bis SO 12.00 bis 17.00 Uhr"

In [ ]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Trickfilm-Plakat-Kunst-Arbeiten-Fuer-Den-Ddr-Filmverleih-113199.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"

In [ ]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Panorama-Amazonien-Im-Panometer-Dresden-Dresden-113732.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-31"

In [ ]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Zeitfenster-Spurensuche-Zur-Industriegeschichte-Im-Ernemannbau-Dresden-113361.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-20"

In [ ]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Physics-Of-Life-Physik-Des-Lebens-Dresden-109648.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-09-01"
df.loc[id, "extrEndDate"] = "2024-10-27"

In [ ]:
flt = df["Event_URL"].str.contains('https://sachsen-net.com/veranstaltungen/heute/Dresdner-Herbstvolksfest-Der-Freizeitpark-Dresden-119283.html', na=False)
id = max(df.index) + 1
print(id)
tmp = df[flt]
assert len(df[flt])>0 
df = df[~flt]
tmp = tmp.iloc[0]
print(tmp)
df.loc[id]=tmp
df.loc[id, "isOnMultipleDays"] = True
df.loc[id, "extrStartDate"] = "2024-10-03"
df.loc[id, "extrEndDate"] = "2024-10-31"
df.loc[id, "extrMultipleDaysDetails"] = """ Montag bis Donnerstag 15:00 - 22:00 Uhr
Freitag 15:00 - 23:00 Uhr
Samstag 14:00 - 23:00 Uhr
Sonntag 14:00 - 22:00 Uhr
SCHLIESSTAGE 21.10., 22.10., 28.10., 29.10.2024"""
df.loc[id, "extrLocationName"]="Volksfestgelände Pieschener Allee"
df.loc[id, "extrLocationDetails"]="01067 Dresden Pieschener Allee 1"

### extra scraped info from browse ai

In [ ]:
len(df)

In [270]:
df.loc[:, "count_Event_URL"]= df.Event_URL.map(df.Event_URL.value_counts())

In [ ]:
df.query("count_Event_URL>1")[["isOnMultipleDays", "Event_URL"]].value_counts()

In [ ]:
len(df)

In [273]:
# df_urls.to_csv(f"{DATA_FOLDER}urls_to_check.csv", index=False)

In [274]:
df_scraped = pd.read_csv(f"{DATA_FOLDER}urls_to_check_scraped.csv")

In [275]:
for index, row in df_scraped.iterrows():
    has_time=False
    url = row["Origin URL"]
    dates_scraped = row["dates"]
    location_details = row["location_details"]
    location_name = row["location_name"]

    if "Flughafen" in dates_scraped:
        dates_possibly_with_time=dates_scraped.split(",")[0]
    elif "Dienstag" in dates_scraped:
        dates_possibly_with_time=dates_scraped.split(",")[0]
    else:
        split = dates_scraped.split(" - ")
        assert len(split)==2, split
        dates_possibly_with_time = split[0]

    if "," in dates_possibly_with_time:
        dates, times = dates_possibly_with_time.split(",")[0], dates_possibly_with_time.split(",")[1]
        has_time=True


    start_date = pd.to_datetime(dates.split(" bis ")[0], dayfirst=True).strftime("%Y-%m-%d")
    end_date = pd.to_datetime(dates.split(" bis ")[1], dayfirst=True).strftime("%Y-%m-%d")
    flt = df["Event_URL"]==url

    assert len(df.loc[flt]) > 0, url

    df.loc[flt, "extrStartDate"] = start_date
    df.loc[flt, "extrEndDate"] = end_date
    df.loc[flt, "isOnMultipleDays"] = True

    if has_time:
        split=times.split(" bis ")
        assert len(split)==2, split
        start_time = split[0].replace(".00", "")
        end_time = split[1].replace(" Uhr", "").replace(".00", "")
        df.loc[flt, "extrStartTime"] = start_time
        df.loc[flt, "extrEndTime"] = end_time

    
    
    
    
    

In [ ]:
len(df)

# final checks and keep only suitable observations

In [ ]:
len(df)

In [278]:
extracted_columns = []
for col in df.columns:
    if col.startswith("extr"):
        extracted_columns.append(col)
    
extracted_columns.append("isOnMultipleDays")

In [ ]:
df.extrEndDate.isna().value_counts()

In [ ]:
for col in extracted_columns:
    print(df[col].isna().value_counts())


In [ ]:
flt = (df.isOnMultipleDays==False ) & (df.extrStartDate!="nan") & (df.extrStartTime!="nan") &\
    ( (df.extrLocationName!="nan") | (df.extrLocationDetails!="nan") | (df.extrStreetHouseNumber!="nan")) 
len(df[flt])

In [ ]:
df.loc[flt, extracted_columns]

## save

In [283]:
exclude_columns= ['Date (GMT)', 'Job Link']

subset_columns = [col for col in df.columns if col not in exclude_columns]

df = df.drop_duplicates(subset=subset_columns)

In [ ]:
len(df)

In [285]:
df = df.dropna(how="all", axis=1)

In [286]:
# df.to_excel(f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx")

In [287]:
df.to_csv(f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv")

In [288]:
df.to_pickle(f"{DATA_FOLDER}df_browse_ai {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")